### Let's train a tensorflow model for sentiment analysis ###

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Load IMDB reviews dataset
num_words = 10000  
max_length = 200  
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Pad sequences to a fixed length
x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)

# Create a simple classification model
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_length))
model.add(Flatten()) 
model.add(Dense(1, activation='sigmoid'))  
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train for 10 epochs
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


2023-11-03 15:30:16.891572: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


17464789/17464789 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           320000    
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 1)                 6401      
                                                                 
Total params: 326401 (1.25 MB)
Trainable params: 326401 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
157/157 [==============================] - 2s 9ms/step - loss: 0.6337 - accuracy: 0.6539 - val_loss: 0.4574 - val_accuracy: 0.8240
Epoch 2/10
157/157 [==============================] - 1s 9ms/step - loss: 0.3

In [3]:
model.save("sentiment_analysis_model.h5")

/Users/hpiercehoffman/code/venv/nitro/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Now we'll serve the model ###

In [17]:
from fastapi import FastAPI
from typing import List
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

app = FastAPI()
model = tf.keras.models.load_model('sentiment_analysis_model.h5')
max_length = 200

@app.post("/predict")
def predict_sentiment(reviews: List[str]):
    
    # Tokenize and pad input sequences
    tokenizer = tf.keras.datasets.imdb.get_word_index()
    reviews_sequences = [[tokenizer.get(word, 0) for word in review.split()] for review in reviews]
    reviews_sequences = pad_sequences(reviews_sequences, maxlen=max_length)
    
    # Make predictions
    predictions = model.predict(reviews_sequences)
    sentiment_predictions = [{'sentiment': 'positive' if pred >= 0.5 else 'negative', 'confidence': float(pred)} for pred in predictions]
    return sentiment_predictions


### Save the tokenizer ###

In [19]:
tokenizer = tf.keras.datasets.imdb.get_word_index()

In [21]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### We will now test the model ###

In [1]:
import requests
import json

payload = [
    "This movie was fantastic! I loved every moment of it.",
    "The acting was terrible, and the plot made no sense.",
    "Average movie, not great but not terrible either." 
]

response = requests.post("http://localhost:8888/predict", json=payload)

# Unpack and access the response content
if response.status_code == 200:
    sentiment_predictions = response.json()
    for prediction in sentiment_predictions:
        print(f"Sentiment: {prediction['sentiment']}, Confidence: {prediction['confidence']:.2f}")
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)  

Sentiment: positive, Confidence: 0.58
Sentiment: positive, Confidence: 0.78
Sentiment: positive, Confidence: 0.67
